# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import sqlite3
import pandas as pd

conn = sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [2]:
# Your code here
pd.read_sql("""
    SELECT firstName, lastName
    FROM employees AS e
        JOIN offices AS o
        USING(officeCode)
    WHERE city == "Boston"
""", conn)

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [3]:
# Your code here
pd.read_sql("""
    SELECT officeCode, city, COUNT() AS num_employees
    FROM employees
        JOIN offices
        USING(officeCode)
    GROUP BY city
    ORDER BY num_employees ASC
""", conn)

,officeCode,city,num_employees
0,2,Boston,2
1,7,London,2
2,3,NYC,2
3,5,Tokyo,2
4,6,Sydney,4
5,4,Paris,5
6,1,San Francisco,6


## Write 3 questions of your own and answer them

In [4]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

'\nHow many customers are there per office?\n'

In [5]:
"""
What employee has the most customers?
"""

# Your code here
pd.read_sql("""
    SELECT e.firstName || " " || e.lastName AS employee_name, COUNT() AS number_of_customers
    FROM customers AS c
        JOIN employees AS e
        ON c.salesRepEmployeeNumber = e.employeeNumber
    GROUP BY c.salesRepEmployeeNumber
    ORDER BY number_of_customers DESC
""", conn)

,employee_name,number_of_customers
0,Pamela Castillo,10
1,Barry Jones,9
2,Larry Bott,8
3,George Vanauf,8
4,Gerard Hernandez,7
5,Foon Yue Tseng,7
6,Martin Gerard,6
7,Loui Bondur,6
8,Steve Patterson,6
9,Julie Firrelli,6


In [6]:
"""
What customers have orders in process?
"""

# Your code here
pd.read_sql("""
    SELECT c.customerName, COUNT() AS orders_in_process
    FROM orders AS o
        JOIN customers AS c
        USING(customerNumber)
    WHERE o.status == "In Process"
    GROUP BY customerNumber
""", conn)

,customerName,orders_in_process
0,La Rochelle Gifts,1
1,Mini Gifts Distributors Ltd.,1
2,Euro+ Shopping Channel,1
3,Diecast Classics Inc.,1
4,Souveniers And Things Co.,1
5,Petit Auto,1


In [7]:
"""
What product Line had the most orders?
"""

# Your code here
pd.read_sql("""
    SELECT p.productLine, SUM(o.quantityOrdered) AS quantity_ordered
    FROM orderdetails AS o
        JOIN products AS p
        USING(productCode)
    GROUP BY p.productLine
    ORDER BY quantity_ordered DESC
    LIMIT 1
""", conn)

,productLine,quantity_ordered
0,Classic Cars,35582


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [8]:
# Your code here
pd.read_sql("""
    SELECT DISTINCT e.firstName || " " || e.lastName AS employee, p.productName
    FROM employees AS e
        JOIN customers AS c
        ON e.employeeNumber = c.salesRepEmployeeNumber
            JOIN orders AS o
            USING(customerNumber)
                JOIN orderdetails AS od
                USING(orderNumber)
                    JOIN products AS p
                    USING(productCode)    
""", conn)

,employee,productName
0,Leslie Jennings,1958 Setra Bus
1,Leslie Jennings,1940 Ford Pickup Truck
2,Leslie Jennings,1939 Cadillac Limousine
3,Leslie Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie Jennings,1968 Ford Mustang
...,...,...
1363,Martin Gerard,1958 Chevy Corvette Limited Edition
1364,Martin Gerard,1992 Porsche Cayenne Turbo Silver
1365,Martin Gerard,1954 Greyhound Scenicruiser
1366,Martin Gerard,1950's Chicago Surface Lines Streetcar


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [9]:
# Your code here
pd.read_sql("""
    SELECT e.firstName || " " || e.lastName AS employee, SUM(od.quantityOrdered) AS products_sold
    FROM employees AS e
        JOIN customers AS c
        ON e.employeeNumber = c.salesRepEmployeeNumber
            JOIN orders AS o
            USING(customerNumber)
                JOIN orderdetails AS od
                USING(orderNumber)
    GROUP BY e.employeeNumber
""", conn)

,employee,products_sold
0,Leslie Jennings,11854
1,Leslie Thompson,4056
2,Julie Firrelli,4227
3,Steve Patterson,5561
4,Foon Yue Tseng,5016
5,George Vanauf,7423
6,Loui Bondur,6186
7,Gerard Hernandez,14231
8,Pamela Castillo,9290
9,Larry Bott,8205


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [17]:
# Your code here
pd.read_sql("""
    SELECT e.firstName || " " || e.lastName AS employee, COUNT(od.productCode) AS different_products_sold
    FROM employees AS e
        JOIN customers AS c
        ON e.employeeNumber = c.salesRepEmployeeNumber
            JOIN orders AS o
            USING(customerNumber)
                JOIN orderdetails AS od
                USING(orderNumber)
    GROUP BY e.employeeNumber
    HAVING different_products_sold > 200
""", conn)

,employee,different_products_sold
0,Leslie Jennings,331
1,George Vanauf,211
2,Gerard Hernandez,396
3,Pamela Castillo,272
4,Larry Bott,236
5,Barry Jones,220


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!